# Isomatrix Tools

> Tools for converting isomatrix files into anndata objects for integration with the Scanpy ecosystem.


In [1]:
#| default_exp isomatrix_tools

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
import pandas as pd
import scanpy as sc
from scanpy import AnnData
from scipy.sparse import csr_matrix
import warnings

def isomatrix_to_anndata(file_path:str,  # The path to the isomatrix csv file to be read.
                        sparse:bool=True  # Flag to determine if the output should be a sparse matrix.
) -> AnnData: # The converted isomatrix as a scanpy compatible  anndata object
    """
    This function converts an isomatrix txt file (SiCeLoRe output) into an AnnData object compatible with scanpy

    """
    
    # Read in the data from the file
    df = pd.read_csv(file_path, sep='\t', index_col=0)
    # Filter out rows where the transcriptId is "undef"
    df = df.loc[df["transcriptId"] != "undef"]
    
    df = df.reset_index()
    df = df.transpose()
    
    # Extract the rows with 'gene_id', 'transcript_id', 'nb_exons' from the DataFrame
    additional_info_rows = df.loc[df.index.intersection(['geneId', 'transcriptId', 'nbExons'])]
    # Drop 'gene_id', 'transcript_id', 'nb_exons' rows from the DataFrame if they exist
    df = df.drop(['geneId', 'transcriptId', 'nbExons'], errors='ignore')

    # Convert the DataFrame to a sparse matrix if the sparse flag is True
    if sparse:
        matrix = csr_matrix(df.values.astype('float32'))
    else:
        try:
            matrix = df.values.astype('float32')
        except ValueError:
            print("Error: Non-numeric data present in the DataFrame. Cannot convert to float.")
            return None
    
    # Convert the matrix to an AnnData object
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        anndata = sc.AnnData(X=matrix, obs=pd.DataFrame(index=df.index), var=pd.DataFrame(index=df.columns))
    
    # Add additional information to the AnnData object vars
    for info in ['geneId', 'transcriptId', 'nbExons']:
        if info in additional_info_rows.index:
            anndata.var[info] = additional_info_rows.loc[info, :].values
            if info == 'nbExons':
                anndata.var[info] = anndata.var[info].astype('int32')
    
    return anndata

In [4]:
#| export
def download_test_data() -> str: #The absolute path of the extracted file 'sample_isomatrix.txt' if the download is successful.
    """
    This function downloads a test data file from a specified URL, saves it locally, and extracts it.
    """
    import urllib.request
    import gzip
    import shutil
    import os

    # URL of the file to be downloaded
    url = "https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3748nnn/GSM3748087/suppl/GSM3748087%5F190c.isoforms.matrix.txt.gz"

    # Download the file from `url` and save it locally under `file.txt.gz`:
    urllib.request.urlretrieve(url, 'file.txt.gz')

    # Check if the file is downloaded correctly
    if os.path.exists('file.txt.gz'):
        print("File downloaded successfully")
        # Now we need to extract the file
        with gzip.open('file.txt.gz', 'rb') as f_in:
            with open('sample_isomatrix.txt', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print("File extracted successfully")
        return os.path.abspath('sample_isomatrix.txt')
    else:
        print("Failed to download the file")
        return None


Example usage of `isomatrix_to_anndata`: We can use the `download_test_data` function to download a small isoform matrix dataset for demonstrating the functionality.


In [8]:
from longreadtools.isomatrix_tools import * 
test_file = download_test_data() 

File downloaded successfully
File extracted successfully


In [9]:
anndata = isomatrix_to_anndata(test_file)

Lets take a look at the anndata object generated from the isomatrix.

In [10]:
anndata

AnnData object with n_obs × n_vars = 190 × 20834
    var: 'geneId', 'transcriptId'

In [11]:
#| hide
def test_isomatrix_to_anndata():
    # Test with a known file
    test_file = download_test_data()
    anndata = isomatrix_to_anndata(test_file)

    # Check the type of the returned object
    assert isinstance(anndata, sc.AnnData), "The returned object is not an AnnData object."

    # Check the dimensions of the AnnData object
    assert anndata.shape == (190, 20834), "The dimensions of the AnnData object are not as expected."

    # Check the var names of the AnnData object
    assert 'geneId' in anndata.var, "The 'geneId' is not in the var of the AnnData object."
    assert 'transcriptId' in anndata.var, "The 'transcriptId' is not in the var of the AnnData object."




Often, it may be necessary to convert more than one isomatrix in bulk. The function `multiple_isomatrix_conversion` has been designed for this purpose. It leverages Python's multiprocessing capabilities to perform this task in a fast and efficient manner.


In [12]:
#| hide
import os
import re

directory = '/data/analysis/data_mcandrew/000-sclr-discovair/'

# Define the regular expression pattern
pattern = re.compile('.*(_BIOP_INT|BIOP_NAS)$')

# Get a list of all files in the directory
all_files = os.listdir(directory)

# Filter the list to include only files that match the pattern
matching_files = [os.path.join(directory, f) for f in all_files if pattern.match(f)]

# Print the list of matching files
print(matching_files)

individual_runs = matching_files
individual_runs = [f'{run}_isomatrix.txt' for run in individual_runs]
individual_runs = [os.path.join(run, f'{os.path.basename(run)}_isomatrix.txt') for run in matching_files]




['/data/analysis/data_mcandrew/000-sclr-discovair/D498_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D492_BIOP_NAS', '/data/analysis/data_mcandrew/000-sclr-discovair/D494_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D500_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D494_BIOP_NAS', '/data/analysis/data_mcandrew/000-sclr-discovair/D496_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D499_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D493_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D493_BIOP_NAS', '/data/analysis/data_mcandrew/000-sclr-discovair/D534_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D490_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D500_BIOP_NAS', '/data/analysis/data_mcandrew/000-sclr-discovair/D495_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D492_BIOP_INT']


In [19]:
#| export

from multiprocessing import Pool
import os
from functools import partial

def multiple_isomatrix_conversion(file_paths: list, # A list of file paths to be converted.
                                  verbose: bool = False # If True, print progress messages.
                                  ):
    """
    This function takes a list of file paths, converts each file from isomatrix to anndata format, 
    and saves the converted file in the same location with the same name but with a .h5ad extension.
    """
    
    def convert_and_save_file(sample, verbose):
        anndata = isomatrix_to_anndata(sample)
        anndata.write_h5ad(sample.replace('.txt', '.h5ad'))
        if verbose:
            print(f"File {sample.replace('.txt', '.h5ad')} was successfully written to disk.")

    with Pool() as p:
        p.map(partial(convert_and_save_file, verbose=verbose), file_paths)



Here is an example of how to use the function to convert several Isomatrix objects at once. The input is a list of paths to the Isomatrix text files.


In [20]:
multiple_isomatrix_conversion(individual_runs, verbose=True)

File /data/analysis/data_mcandrew/000-sclr-discovair/D498_BIOP_INT/D498_BIOP_INT_isomatrix.h5ad was successfully written to disk.
File /data/analysis/data_mcandrew/000-sclr-discovair/D500_BIOP_NAS/D500_BIOP_NAS_isomatrix.h5ad was successfully written to disk.
File /data/analysis/data_mcandrew/000-sclr-discovair/D500_BIOP_INT/D500_BIOP_INT_isomatrix.h5ad was successfully written to disk.
File /data/analysis/data_mcandrew/000-sclr-discovair/D493_BIOP_NAS/D493_BIOP_NAS_isomatrix.h5ad was successfully written to disk.
File /data/analysis/data_mcandrew/000-sclr-discovair/D494_BIOP_NAS/D494_BIOP_NAS_isomatrix.h5ad was successfully written to disk.
File /data/analysis/data_mcandrew/000-sclr-discovair/D493_BIOP_INT/D493_BIOP_INT_isomatrix.h5ad was successfully written to disk.
File /data/analysis/data_mcandrew/000-sclr-discovair/D499_BIOP_INT/D499_BIOP_INT_isomatrix.h5ad was successfully written to disk.
File /data/analysis/data_mcandrew/000-sclr-discovair/D494_BIOP_INT/D494_BIOP_INT_isomatrix

In [15]:
#| hide
import nbdev; nbdev.nbdev_export()